In [53]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import numpy as np
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [91]:
df = pd.read_csv('data.csv')
df

,SALE_DT,STORE,SALES
0,2019-01-01 00:00:00.000,Store_a,0.00
1,2019-01-02 00:00:00.000,Store_a,16553359.58
2,2019-01-03 00:00:00.000,Store_a,17956624.58
3,2019-01-04 00:00:00.000,Store_a,19440652.23
4,2019-01-05 00:00:00.000,Store_a,20677704.73
...,...,...,...
1272,2022-06-26 00:00:00.000,Store_a,17243093.78
1273,2022-06-27 00:00:00.000,Store_a,13538082.87
1274,2022-06-28 00:00:00.000,Store_a,13565274.83
1275,2022-06-29 00:00:00.000,Store_a,14959597.22


In [92]:
df.isnull().sum()

SALE_DT    0
STORE      0
SALES      0
dtype: int64

In [102]:
df.columns = ['date', 'store','sales']
df.head()

,date,store,sales
0,2019-01-01,Store_a,0.00
1,2019-01-02,Store_a,16553359.58
2,2019-01-03,Store_a,17956624.58
3,2019-01-04,Store_a,19440652.23
4,2019-01-05,Store_a,20677704.73


In [121]:
# Filter records for store 1 and item 1 -> to be able to scale to other items in the future
# df = df[df['store'] == 1]
# df = df[df['item'] == 1]

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d') # convert date column to datatime object

# # Create Date-related Features to be used for EDA and Supervised ML: Regression
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['weekday'] = np.where(df.weekday == 0, 7, df.weekday)
#
# # Split the series to predict the last 3 months of 2017
temp_df = df.set_index('date')
train_df = temp_df.loc[:'2022-02-01'].reset_index(drop=False)
test_df = temp_df.loc['2022-10-01':].reset_index(drop=False)
#
df

,date,store,sales,year,month,day,weekday
0,2019-01-01,Store_a,0.00,2019,1,1,1
1,2019-01-02,Store_a,16553359.58,2019,1,2,2
2,2019-01-03,Store_a,17956624.58,2019,1,3,3
3,2019-01-04,Store_a,19440652.23,2019,1,4,4
4,2019-01-05,Store_a,20677704.73,2019,1,5,5
...,...,...,...,...,...,...,...
1272,2022-06-26,Store_a,17243093.78,2022,6,26,6
1273,2022-06-27,Store_a,13538082.87,2022,6,27,7
1274,2022-06-28,Store_a,13565274.83,2022,6,28,1
1275,2022-06-29,Store_a,14959597.22,2022,6,29,2


In [122]:
train_df.head()

,date,store,sales,year,month,day,weekday
0,2019-01-01,Store_a,0.00,2019,1,1,1
1,2019-01-02,Store_a,16553359.58,2019,1,2,2
2,2019-01-03,Store_a,17956624.58,2019,1,3,3
3,2019-01-04,Store_a,19440652.23,2019,1,4,4
4,2019-01-05,Store_a,20677704.73,2019,1,5,5


In [108]:
test_df.head()

,date,store,sales,year,month,day,weekday
0,2019-01-01,Store_a,0.00,2019,1,1,1
1,2019-01-02,Store_a,16553359.58,2019,1,2,2
2,2019-01-03,Store_a,17956624.58,2019,1,3,3
3,2019-01-04,Store_a,19440652.23,2019,1,4,4
4,2019-01-05,Store_a,20677704.73,2019,1,5,5


ValueError: min() arg is an empty sequence